In [14]:
import os
import pandas as pd
import nltk
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.ensemble import RandomForestClassifier
from imblearn.over_sampling import ADASYN, SMOTE, RandomOverSampler
import sklearn.linear_model as model
import csv
from sklearn.feature_extraction.text import TfidfVectorizer

### Basic Logistic Regression

Use Michelle's method for importing data since I can't do it correctly:

In [2]:
train_data = np.genfromtxt('Data/training_data.txt',dtype='str')
test_data = np.genfromtxt('Data/test_data.txt',dtype='str')
#train_data = np.loadtxt('training_data.txt',  delimiter=' ')
#test_data = np.loadtxt('test_data.txt')
train_labels = train_data[0, :]
train_stars = train_data[1:, 0]
train_reviews = train_data[1:, 1:]

test_labels = test_data[0, :]
#test_stars = train_data[1:, 0]
test_reviews = test_data[1:, 0:]

trainX = train_reviews
trainY = train_stars
testX = test_reviews
#testY = test_stars

print("trainX = ", trainX.shape)
print("trainY = ", trainY.shape)
print("testX = ", testX.shape)


for i in range(0, len(trainX)):
    for j in range(0, len(trainX[0])):
        trainX[i][j] = int(trainX[i][j])

for i in range(0, len(testX)):
    for j in range(0, len(testX[0])):
        testX[i][j] = int(testX[i][j])

for i in range(0, len(trainY)):
        trainY[i] = int(trainY[i])
#for i in range(0, len(trainY)):
        #testY[i] = int(testY[i])

trainX =  (20000, 1000)
trainY =  (20000,)
testX =  (10000, 1000)


Define a function for classification error:

In [3]:
def class_error(y, y_pred):
    misclassified = 0
    for i in range(len(y)):
        if y[i] != y_pred[i]:
            misclassified += 1
    return float(misclassified) / len(y)

Split data into 80% training set and 20% test set.

In [4]:
num_rows = []
for i in range(len(trainY)):
    num_rows.append(i)
    
np.random.shuffle(num_rows)
train_indices = num_rows[:16000]
valid_indices = num_rows[16000:]

train_X = []
train_y = []
valid_X = []
valid_y = []

for index in train_indices:
    train_X.append(trainX[index])
    train_y.append(trainY[index])
    
for index in valid_indices:
    valid_X.append(trainX[index])
    valid_y.append(trainY[index])

With Michelle's code, it doesn't actually convert the array contents into integers because she tried to do it in place. This code does it correctly out of place:

In [5]:
train_X2 = []
for row in train_X:
    temp = []
    for element in row:
        temp.append(int(element))
    train_X2.append(temp)

valid_X2 = []
for row in valid_X:
    temp = []
    for element in row:
        temp.append(int(element))
    valid_X2.append(temp)
    
valid_y2 = []
for row in valid_y:
    valid_y2.append(int(row))
    
train_y2 = []
for row in train_y:
    train_y2.append(int(row))
    
test_X2 = []
for row in testX:
    temp = []
    for element in row:
        temp.append(int(element))
    test_X2.append(temp)

Convert everything back to a numpy array:

In [6]:
train_X2 = np.array(train_X2)
valid_X2 = np.array(valid_X2)
train_y2 = np.array(train_y2)
valid_y2 = np.array(valid_y2)
test_X2 = np.array(test_X2)

Fit the model:

In [7]:
logistic = model.LogisticRegression()
logistic.fit(train_X2, train_y2)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

Print the classification error on the training set:

In [8]:
class_error(logistic.predict(train_X2), train_y2)

0.122625

Print the classification error on the validation set:

In [9]:
class_error(logistic.predict(valid_X2), valid_y2)

0.1525

Logistic regression works better than a random forest. Now, we want to combine the training and validation sets again so that we can train our model on all the data to prevent overfitting. We will then use this to predict on the test set:

In [10]:
all_X = np.concatenate((train_X2, valid_X2))
all_y = np.concatenate((train_y2, valid_y2))

In [11]:
logistic = model.LogisticRegression()
logistic.fit(all_X, all_y)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

A function for writing to the text file for the submission:

In [41]:
def writeToText(predictions):
    array = [["Id","Prediction"]]
    index = 1
    for i in range(len(predictions)):
        predict = predictions[index - 1]
        array.append([index, int(predictions[index - 1])])
        index += 1
    f = open("test_logistic_prediction_TFIDF.txt", 'w')
    writer = csv.writer(f, delimiter=',', quotechar='|')
    writer.writerows(array)

In [13]:
writeToText(logistic.predict(test_X2))

### Logistic Regression with TF-IDF

A function to convert the set into a TF-IDF matrix:

In [36]:
def TFIDF(X, labels):
    documents = []

    for element in X:
        temp = ''
        for i in range(len(element)):
            word = labels[i + 1] + ' '
            new_word = word * element[i]
            temp += (new_word)
        documents.append(temp)
        
    tokenize = lambda doc: doc.lower().split(" ")
    sklearn_tfidf = TfidfVectorizer(norm='l2',min_df=0, use_idf=True, smooth_idf=False, sublinear_tf=True, tokenizer=tokenize)
    return sklearn_tfidf.fit_transform(documents)

Fit the model using the training data:

In [38]:
logistic = model.LogisticRegression()
logistic.fit(TFIDF(train_X2, train_labels), train_y2)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

In [40]:
class_error(logistic.predict(TFIDF(valid_X2, train_labels)), valid_y2)

0.14525

This model has a lower classification error than the basic regression. We will now undergo the same procedure as in the last section to get the test predictions:

In [43]:
logistic = model.LogisticRegression()
logistic.fit(TFIDF(all_X, train_labels), all_y)
writeToText(logistic.predict(TFIDF(test_X2, train_labels)))